In [1]:
# Import necessary libraries
import logging
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import NotFoundError

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# Define Elasticsearch client
es = Elasticsearch(["http://localhost:9200"])  # Adjust the URL if your Elasticsearch is hosted elsewhere

def create_index_with_dense_vector(index_name):
    mapping = {
        "mappings": {
            "properties": {
                "id": {"type": "keyword"},
                "link": {"type": "text"},
                "title": {"type": "text"},
                "summary": {"type": "text"},
                "nlp_processed": {"type": "boolean"},
                "entities": {"type": "nested", "properties": {"text": {"type": "text"}, "label": {"type": "keyword"}}},
                "sentiment": {"type": "float"},
            }
        }
    }
    
    try:
        es.indices.create(index=index_name, body=mapping, ignore=400)
        logger.info(f"Created index with dense_vector mapping: {index_name}")
    except Exception as e:
        logger.error(f"Error creating index {index_name}: {e}")
        raise

def reindex_data(old_index, new_index):
    try:
        es.reindex(
            body={
                "source": {"index": old_index},
                "dest": {"index": new_index},
            },
            wait_for_completion=True
        )
        logger.info("Reindexed data from old index to new index.")
    except Exception as e:
        logger.error(f"Error reindexing data: {e}")
        raise

def delete_index(index_name):
    try:
        es.indices.delete(index=index_name, ignore=[400, 404])
        logger.info(f"Deleted index: {index_name}")
    except NotFoundError:
        logger.warning(f"Index {index_name} does not exist.")
    except Exception as e:
        logger.error(f"Error deleting index {index_name}: {e}")
        raise

# # Define index names
# old_index = "rss_feeds"
# new_index = "rss_feeds_v2"

# # Execute the operations
# create_index_with_dense_vector(new_index)  # Create a new index with dense_vector mapping
# reindex_data(old_index, new_index)  # Reindex data from old to new index
# delete_index(old_index)  # Delete the old index after reindexing

# # Confirm that the new index was created and has the correct mapping
# mapping = es.indices.get_mapping(index=new_index)
# print(f"Mapping for index '{new_index}':")
# print(mapping)


In [2]:
def rename_index(old_index, new_index):
    try:
        # Create an alias to point to the new index
        es.indices.put_alias(index=new_index, name=old_index)
        logger.info(f"Renamed index '{new_index}' to '{old_index}' using alias.")
    except Exception as e:
        logger.error(f"Error renaming index {new_index} to {old_index}: {e}")
        raise
# rename_index(old_index, new_index)  # Rename (alias) new index to old index name

# # Confirm that the new index is now used as the original
# aliases = es.indices.get_alias(index=old_index)
# print(f"Aliases for index '{old_index}':")
# print(aliases)

In [3]:
# Define index names
old_index = "rss_feeds"
# new_index = "rss_feeds_v2"

delete_index(old_index)  # Delete the old index after reindexing
# delete_index(new_index)  # Delete the new index after renaming
# create_index_with_dense_vector(old_index)

/tmp/ipykernel_386314/2317237416.py:51: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index=index_name, ignore=[400, 404])
2024-09-06 22:01:14,871 - INFO - DELETE http://localhost:9200/rss_feeds [status:200 duration:0.071s]
2024-09-06 22:01:14,872 - INFO - Deleted index: rss_feeds
